In [123]:
### PREREQUISTIES
### (many used only in one notebook...)

import os
from importlib import reload
import pandas as pd
import numpy as np
import pickle
import math
import random
import sys
import csv
import unicodedata
import requests
import re
import networkx as nx
from scipy import stats
import plotly
import kaleido

import nltk

import matplotlib.pyplot as plt
import seaborn as sns

# gensim parts
from gensim import corpora
from gensim import models

### scikit-learn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.decomposition import TruncatedSVD 
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import euclidean_distances

import plotly.graph_objects as go

import sddk

import gspread
from gspread_dataframe import get_as_dataframe, set_with_dataframe
from google.oauth2 import service_account # based on google-auth library

In [124]:
import textnet

In [125]:
reload(textnet)

<module 'textnet' from '/home/kasev/ECCE_AGT/scripts/textnet.py'>

In [126]:
### not neccessary for reading the data, just for exporting them to sciencedata.dk
conf = sddk.configure("SDAM_root", "648597@au.dk")

sciencedata.dk username (format '123456@au.dk'): 648597@au.dk
sciencedata.dk password: ········
connection with shared folder established with you as its owner
endpoint variable has been configured to: https://sciencedata.dk/files/SDAM_root/


In [127]:
# to access gsheet, you need Google Service Account key json file
# I have mine located in my personal space on sciencedata.dk, so I read it from there:

# (1) read the file and parse its content
file_data = conf[0].get("https://sciencedata.dk/files/ServiceAccountsKey.json").json()
# (2) transform the content into crendentials object
credentials = service_account.Credentials.from_service_account_info(file_data)
# (3) specify your usage of the credentials
scoped_credentials = credentials.with_scopes(['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive'])
# (4) use the constrained credentials for authentication of gspread package
gc = gspread.Client(auth=scoped_credentials)

ECCE_AGT_overview = gc.open_by_url("https://docs.google.com/spreadsheets/d/1KPpPaeX215HR_fVrakvJp8aB6oZDhHFTcBw0MKLw6as/edit?usp=sharing")

In [130]:
AGT = sddk.read_file("SDAM_data/AGT/AGT_lemmatized_20201211.json", "df", conf)
AGT.head(5)

,filename,author,title,wordcount,author_id,doc_id,raw_date,date_avr,date_probs,date_manual,provenience,tlg_epithet,clean_string,n_sentences,lemmatized_sentences
0,tlg0001.tlg001.perseus-grc2.xml,Apollonius Rhodius,Argonautica,38822,tlg0001,tlg0001.tlg001,3 B.C.,-2.5,{'-2.5': 1},-2.5,pagan,Epici/-ae,"ἀρχόμενος σέο, Φοῖβε, παλαιγενέων κλέα φωτῶν μ...",3252,"[[ἄρχω, Φοῖβος, παλαιγενής, κλέος, φώς, μιμνήσ..."
1,tlg0003.tlg001.perseus-grc2.xml,Thucydides,The Peloponnesian War,150118,tlg0003,tlg0003.tlg001,5 B.C.,-4.5,{'-4.5': 1},-4.5,pagan,Historici/-ae,Θουκυδίδης Ἀθηναῖος ξυνέγραψε τὸν πόλεμον τῶν...,6068,"[[θουκυδίδης, Ἀθηναῖος, συγγράφω, πόλεμος, Πελ..."
2,tlg0004.tlg001.perseus-grc1.xml,Diogenes Laertius,Lives of Eminent Philosophers,110763,tlg0004,tlg0004.tlg001,A.D. 3,2.5,{'2.5': 1},NaN,,Biographi,Τὸ τῆς φιλοσοφίας ἔργον ἔνιοί φασιν ἀπὸ βαρβάρ...,10245,"[[φιλοσοφία, ἔργον, ἔνιοι, φημί, βάρβαρος, ἄρχ..."
3,tlg0005.tlg001.perseus-grc1.xml,Theocritus,Idylls,19200,tlg0005,tlg0005.tlg001,4-3 B.C.,-3.0,"{'-3.5': 0.5, '-2.5': 0.5}",NaN,,Bucolici,"̔Αδύ τι τὸ ψιθύρισμα καὶ ἁ πίτυς αἰπόλε τήνα,...",1982,"[[αδύ, ψιθύρισμα, πίτυς, αἰπόλος, τῆνος, πηγή,..."
4,tlg0005.tlg002.perseus-grc1.xml,Theocritus,Epigrams,1734,tlg0005,tlg0005.tlg002,4-3 B.C.,-3.0,"{'-3.5': 0.5, '-2.5': 0.5}",NaN,,Bucolici,τὰ ῥόδα τὰ δροσόεντα καὶ ἁ κατάπυκνος ἐκείνα ἕ...,152,"[[ῥόδον, δροσόεντα, κατάπυκνος, ἐκεῖνος, ἕρπυλ..."


In [131]:
def flat_lemmata(sentences):
    lemmata = [word for sent in sentences for word in sent]
    return lemmata
AGT["lemmata"] = AGT["lemmatized_sentences"].apply(flat_lemmata)

AGT["lemmata_wordcount"] = AGT.apply(lambda row: len(row["lemmata"]), axis=1)
AGT["lemmata_wordcount"].sum() # previously we had 13925726, then 13713183, 14373580, 14342245

14383627

In [14]:
AGT_C.head(5)

,filename,author,title,wordcount,author_id,doc_id,raw_date,date_avr,date_probs,date_manual,provenience,tlg_epithet,clean_string,n_sentences,lemmatized_sentences,lemmata,lemmata_wordcount
385,tlg0031.tlg001.perseus-grc2.xml,None,New Testament - Matthew,18288,tlg0031a,tlg0031.tlg001,A.D. 1,0.5,{'0.5': 1},0.5,christian,[],ΒΙΒΛΟΣ γενέσεως Ἰησοῦ Χριστοῦ υἱοῦ Δαυεὶδ υἱο...,1276,"[[βίβλος, γένεσις, Ἰησοῦς, Χριστός, υἱός, Δαυί...","[βίβλος, γένεσις, Ἰησοῦς, Χριστός, υἱός, Δαυίδ...",8520
386,tlg0031.tlg002.perseus-grc2.xml,None,New Testament - Mark,11274,tlg0031b,tlg0031.tlg002,A.D. 1,0.5,{'0.5': 1},0.5,christian,[],ΑΡΧΗ τοῦ εὐαγγελίου Ἰησοῦ Χριστοῦ . Καθὼς γέγ...,790,"[[ἀρχή, εὐαγγέλιον, Ἰησοῦς, Χριστός], [γράφω, ...","[ἀρχή, εὐαγγέλιον, Ἰησοῦς, Χριστός, γράφω, ἠσα...",5209
387,tlg0031.tlg003.perseus-grc2.xml,Luke (the evangelist),New Testament - Luke,19458,tlg0031luke,tlg0031.tlg003,A.D. 1,0.5,{'0.5': 1},0.5,christian,[],ΕΠΕΙΔΗΠΕΡ ΠΟΛΛΟΙ ἐπεχείρησαν ἀνατάξασθαι διήγ...,1274,"[[πολύς, ἐπιχειρέω, ἀνατάσσομαι, διήγησις, πλη...","[πολύς, ἐπιχειρέω, ἀνατάσσομαι, διήγησις, πληρ...",9070
388,tlg0031.tlg004.perseus-grc2.xml,Johnannine literature,New Testament - John,15590,tlg0031john,tlg0031.tlg004,A.D. 1,0.5,{'0.5': 1},0.5,christian,[],"ΕΝ ΑΡΧΗ ἦν ὁ λόγος, καὶ ὁ λόγος ἦν πρὸς τὸν θ...",1164,"[[εἰμί, ἀρχή, εἰμί, λόγος, λόγος, εἰμί, θεός, ...","[εἰμί, ἀρχή, εἰμί, λόγος, λόγος, εἰμί, θεός, θ...",6989
389,tlg0031.tlg005.perseus-grc2.xml,Luke (the evangelist),New Testament - Acts,18406,tlg0031luke,tlg0031.tlg005,A.D. 1,0.5,{'0.5': 1},0.5,christian,[],"τὸν μὲν πρῶτον λόγον ἐποιησάμην περὶ πάντων, ...",960,"[[πρῶτος, λόγος, ποιέω, πᾶς, Θεόφιλος, ἄρχω, Ἰ...","[πρῶτος, λόγος, ποιέω, πᾶς, Θεόφιλος, ἄρχω, Ἰη...",9033


In [154]:
author = AGT_C["author"][10]
author

'Paul of Tarsus'

In [155]:
sub_df = AGT_C[AGT_C["author_id"]=="tlg0031john"]
sub_df["date_probs"]

388    {'0.5': 1}
407    {'0.5': 1}
408    {'0.5': 1}
409    {'0.5': 1}
Name: date_probs, dtype: object

In [156]:
get_flat_sentences(sub_df["lemmatized_sentences"])

[['εἰμί',
  'ἀρχή',
  'εἰμί',
  'λόγος',
  'λόγος',
  'εἰμί',
  'θεός',
  'θεός',
  'εἰμί',
  'λόγος'],
 ['ἀρχή', 'θεός'],
 ['πᾶς', 'γίγνομαι', 'γίγνομαι', 'εἷς'],
 ['γίγνομαι', 'ζωή', 'εἰμί', 'ζωή', 'φῶς', 'ἄνθρωπος'],
 ['φῶς', 'σκοτία', 'φαίνω', 'σκοτία', 'καταλαμβάνω'],
 ['γίγνομαι', 'ἄνθρωπος', 'ἀποστέλλω', 'θεός', 'ὄνομα', 'ἰωάνης'],
 ['οὗτος', 'ἔρχομαι', 'μαρτυρία', 'μαρτυρέω', 'φῶς', 'πᾶς', 'πιστεύω'],
 ['φῶς', 'μαρτυρέω', 'φῶς'],
 ['φῶς', 'ἀληθινός', 'φωτίζω', 'ἄνθρωπος', 'ἔρχομαι', 'κόσμος'],
 ['κόσμος', 'εἰμί', 'κόσμος', 'γίγνομαι', 'κόσμος', 'γιγνώσκω'],
 ['ἴδιος', 'ἔρχομαι', 'ἴδιος', 'παραλαμβάνω'],
 ['λαμβάνω',
  'δίδωμι',
  'ἐξουσία',
  'τέκνον',
  'θεός',
  'γίγνομαι',
  'πιστεύω',
  'ὄνομα',
  'αἷμα',
  'θέλημα',
  'σάρξ',
  'θέλημα',
  'ἀνήρ',
  'θεός',
  'γεννάω'],
 ['λόγος',
  'σάρξ',
  'γίγνομαι',
  'σκηνόω',
  'θεάομαι',
  'δόξα',
  'δόξα',
  'μονογενής',
  'πατήρ',
  'πλήρης',
  'χάρις',
  'ἀλήθεια'],
 [],
 ['ἰωάνης',
  'μαρτυρέω',
  'κράζω',
  'λέγω',
  'οὗτος',


In [157]:
probs_dict = {}
for d in sub_df["date_probs"]:
    probs_dict.update(d)
probs_dict

{'0.5': 1}

# Comparing All Authors

In [158]:
def grouping_gnt(author_id):
    if author_id == "tlg0031a":
        author_id = "tlg0031matt"
    if author_id == "tlg0031b":
        author_id = "tlg0031mark"
    if re.match("tlg0031\w?$", author_id):
        return "tlg0031rest"
    else:
        return author_id
new_author_ids = AGT["author_id"].apply(grouping_gnt)

AGT["author_id"] = new_author_ids

In [159]:
authors_dicts = []
for author_id in AGT["author_id"].unique():
    sub_df = AGT[AGT["author_id"]==author_id]
    authors_dict = {}
    authors_dict["author_id"] = author_id
    authors_dict["author"] = sub_df["author"][0]
    authors_dict["n_titles"] = len(sub_df)
    authors_dict["titles"] = sub_df["title"].tolist()
    authors_dict["filenames"] = sub_df["filename"].tolist()
    authors_dict["wordcount"] = sub_df["wordcount"].sum()
    authors_dict["lemmata_wordcount"] = sub_df["lemmata_wordcount"].sum()
    authors_dict["n_sentences"] = sub_df["n_sentences"].sum()
    probs_dict = {}
    for d in sub_df["date_probs"]:
        probs_dict.update(d)
    authors_dict["date_probs"] = probs_dict
    authors_dict["date_avr"] = sub_df["date_avr"].tolist()[0]
    authors_dict["provenience"] = sub_df["provenience"].tolist()[0]
    authors_dict["lemmata_lists"] = sub_df["lemmata"].tolist()
    authors_dict["lemmatized_sentences"] = get_flat_sentences(sub_df["lemmatized_sentences"])
    authors_dicts.append(authors_dict)
    
authors_df = pd.DataFrame(authors_dicts)
authors_df.head(5)

,author_id,author,n_titles,titles,filenames,wordcount,lemmata_wordcount,n_sentences,date_probs,date_avr,provenience,lemmata_lists,lemmatized_sentences
0,tlg0001,Apollonius Rhodius,1,[Argonautica],[tlg0001.tlg001.perseus-grc2.xml],38822,23777,3252,{'-2.5': 1},-2.5,pagan,"[[ἄρχω, Φοῖβος, παλαιγενής, κλέος, φώς, μιμνήσ...","[[ἄρχω, Φοῖβος, παλαιγενής, κλέος, φώς, μιμνήσ..."
1,tlg0003,Thucydides,1,[The Peloponnesian War],[tlg0003.tlg001.perseus-grc2.xml],150118,71863,6068,{'-4.5': 1},-4.5,pagan,"[[θουκυδίδης, Ἀθηναῖος, συγγράφω, πόλεμος, Πελ...","[[θουκυδίδης, Ἀθηναῖος, συγγράφω, πόλεμος, Πελ..."
2,tlg0004,Diogenes Laertius,1,[Lives of Eminent Philosophers],[tlg0004.tlg001.perseus-grc1.xml],110763,56872,10245,{'2.5': 1},2.5,,"[[φιλοσοφία, ἔργον, ἔνιοι, φημί, βάρβαρος, ἄρχ...","[[φιλοσοφία, ἔργον, ἔνιοι, φημί, βάρβαρος, ἄρχ..."
3,tlg0005,Theocritus,3,"[Idylls , Epigrams , Syrinx]","[tlg0005.tlg001.perseus-grc1.xml, tlg0005.tlg0...",21011,13016,2137,"{'-3.5': 0.5, '-2.5': 0.5}",-3.0,,"[[αδύ, ψιθύρισμα, πίτυς, αἰπόλος, τῆνος, πηγή,...","[[αδύ, ψιθύρισμα, πίτυς, αἰπόλος, τῆνος, πηγή,..."
4,tlg0006,Euripides,17,"[Cyclops, Ἡρακλεῖδαι, Ἱππόλυτος, Ἀνδρομάχη, Ἑκ...","[tlg0006.tlg001.perseus-grc2.xml, tlg0006.tlg0...",134129,81439,14482,{'-4.5': 1},-4.5,pagan,"[[Βρόμιος, ἔχω, πόνος, χὥτʼ, ἥβη, ἐμός, εὐσθεν...","[[Βρόμιος, ἔχω, πόνος, χὥτʼ, ἥβη, ἐμός, εὐσθεν..."


In [160]:
clean_titles = authors_df["titles"].apply(lambda cell: [title.rpartition(" - ")[2] for title in cell]) 
authors_df["titles"] = clean_titles

In [161]:
def fill_author(author, titles):
    if author == None:
        return titles[0]
    else:
        return author.partition(",")[0]
 
clean_authors = authors_df.apply(lambda row: fill_author(row["author"], row["titles"]), axis=1)
authors_df["author"] = clean_authors

In [162]:
authors_df.loc[authors_df["author_id"]=="tlg0031rest", "author"] = "GNT rest" #, ["author"]] = 


In [163]:
authors_df.loc[authors_df["author_id"]=="tlg1484", "author"] = "Martyrium Polycarpi"
authors_df.loc[authors_df["author_id"]=="tlg1311", "author"] = "Didache" #, ["author"]] = #, ["author"]] = 

In [164]:
authors_df

,author_id,author,n_titles,titles,filenames,wordcount,lemmata_wordcount,n_sentences,date_probs,date_avr,provenience,lemmata_lists,lemmatized_sentences
0,tlg0001,Apollonius Rhodius,1,[Argonautica],[tlg0001.tlg001.perseus-grc2.xml],38822,23777,3252,{'-2.5': 1},-2.5,pagan,"[[ἄρχω, Φοῖβος, παλαιγενής, κλέος, φώς, μιμνήσ...","[[ἄρχω, Φοῖβος, παλαιγενής, κλέος, φώς, μιμνήσ..."
1,tlg0003,Thucydides,1,[The Peloponnesian War],[tlg0003.tlg001.perseus-grc2.xml],150118,71863,6068,{'-4.5': 1},-4.5,pagan,"[[θουκυδίδης, Ἀθηναῖος, συγγράφω, πόλεμος, Πελ...","[[θουκυδίδης, Ἀθηναῖος, συγγράφω, πόλεμος, Πελ..."
2,tlg0004,Diogenes Laertius,1,[Lives of Eminent Philosophers],[tlg0004.tlg001.perseus-grc1.xml],110763,56872,10245,{'2.5': 1},2.5,,"[[φιλοσοφία, ἔργον, ἔνιοι, φημί, βάρβαρος, ἄρχ...","[[φιλοσοφία, ἔργον, ἔνιοι, φημί, βάρβαρος, ἄρχ..."
3,tlg0005,Theocritus,3,"[Idylls , Epigrams , Syrinx]","[tlg0005.tlg001.perseus-grc1.xml, tlg0005.tlg0...",21011,13016,2137,"{'-3.5': 0.5, '-2.5': 0.5}",-3.0,,"[[αδύ, ψιθύρισμα, πίτυς, αἰπόλος, τῆνος, πηγή,...","[[αδύ, ψιθύρισμα, πίτυς, αἰπόλος, τῆνος, πηγή,..."
4,tlg0006,Euripides,17,"[Cyclops, Ἡρακλεῖδαι, Ἱππόλυτος, Ἀνδρομάχη, Ἑκ...","[tlg0006.tlg001.perseus-grc2.xml, tlg0006.tlg0...",134129,81439,14482,{'-4.5': 1},-4.5,pagan,"[[Βρόμιος, ἔχω, πόνος, χὥτʼ, ἥβη, ἐμός, εὐσθεν...","[[Βρόμιος, ἔχω, πόνος, χὥτʼ, ἥβη, ἐμός, εὐσθεν..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
264,tlg7000,"Greek Anthology, Volume V",1,"[Greek Anthology, Volume V]",[tlg7000.tlg001.perseus-grc5.xml],20222,13460,1971,{},NaN,,"[[ἰίαίρβ, θεά, Πάφος, σός, δύναμις, κάλλος, ἀθ...","[[ἰίαίρβ, θεά, Πάφος], [σός, δύναμις, κάλλος, ..."
265,tlg9004,Anonymi In Aristotelis Librum Alterum Analytic...,1,[Anonymi in analyticorum posteriorum librum al...,[tlg9004.tlg001.opp-grc1.xml],21649,8495,2414,{},NaN,,"[[ζητούμενά, ἴσος, ἀριθμός, ὅσος, ἐπίσταμαι, φ...","[[], [ζητούμενά, ἴσος, ἀριθμός, ὅσος, ἐπίσταμα..."
266,tlg9006,Libanius,1,[Libanius Opera],[tlg9006.tlg011.opp-grc1.xml],6745,3175,392,{'12.5': 1},12.5,,"[[ἀνήρ, Λακεδαιμόνιος, ἡγέομαι, κοριν, θίους, ...","[[ἀνήρ, Λακεδαιμόνιος, ἡγέομαι, κοριν, θίους, ..."
267,tlg9010,Suda,1,[Suidae lexicon],[tlg9010.tlg001.1st1K-grc1.xml],589422,292441,139897,{'9.5': 1},9.5,,"[[πάρειμι, βιβλίον, σοῦδα, συνταξάμενοι, ἀνήρ,...","[[πάρειμι, βιβλίον, σοῦδα, συνταξάμενοι, ἀνήρ,..."


In [165]:
len(authors_df)

269

In [166]:
# COUNT δικαιοσύνη IN SENTENCES
def count_term_in_sents(lemmatized_sentences):
    return sum([sent.count("δικαιοσύνη") for sent in lemmatized_sentences])

authors_df["dikaiosyne_N"]  = authors_df["lemmatized_sentences"].apply(count_term_in_sents)

In [167]:
authors_df["dikaiosyne_TF"]  = authors_df.apply(lambda row: row["dikaiosyne_N"] / row["wordcount"], axis=1)
authors_df.sort_values("dikaiosyne_TF", ascending=False, inplace=True)

In [168]:
authors_df

,author_id,author,n_titles,titles,filenames,wordcount,lemmata_wordcount,n_sentences,date_probs,date_avr,provenience,lemmata_lists,lemmatized_sentences,dikaiosyne_N,dikaiosyne_TF
175,tlg1622,Polycarp,1,[Epistula ad Philippenses],[tlg1622.tlg001.1st1K-grc1.xml],1804,951,212,"{'0.5': 0.5, '1.5': 0.5}",1.0,christian,"[[πολύκαρπος, πρέσβυς, ἐκκλησία, θεός, παροικο...","[[πολύκαρπος, πρέσβυς, ἐκκλησία, θεός, παροικο...",8,0.004435
35,tlg0031paul,Paul of Tarsus,7,"[Romans, 1 Corinthians, 2 Corinthians, Galatia...","[tlg0031.tlg006.perseus-grc2.xml, tlg0031.tlg0...",24066,11089,1632,{'0.5': 1},0.5,christian,"[[Παῦλος, δοῦλος, Ἰησοῦς, Χριστός, κλητός, ἀπό...","[[Παῦλος, δοῦλος, Ἰησοῦς, Χριστός, κλητός, ἀπό...",49,0.002036
156,tlg1271,Clemens Romanus,2,"[Epistula I ad Corinthios, Epistula II ad Cori...","[tlg1271.tlg001.1st1K-grc1.xml, tlg1271.tlg002...",13936,6290,1410,{'0.5': 1},0.5,christian,"[[ἐκκλησία, θεός, παροικοῦσα, Ῥώμη, ἐκκλησία, ...","[[ἐκκλησία, θεός, παροικοῦσα, Ῥώμη, ἐκκλησία, ...",21,0.001507
130,tlg0646,Pseudo-Justinus Martyr,1,[Epistula ad Diognetum],[tlg0646.tlg004.1st1K-grc1.xml],2725,1364,240,"{'2.5': 0.5, '4.5': 0.5}",3.5,christian,"[[ὁράω, κράτιστος, διόγνητε, ὑπερεσπουδακότα, ...","[[ὁράω, κράτιστος, διόγνητε, ὑπερεσπουδακότα, ...",4,0.001468
152,tlg1216,Barnabas,1,[The Epistle of Barnabas],[tlg1216.tlg001.perseus-grc2.xml],7440,3450,875,"{'0.5': 0.5, '1.5': 0.5}",1.0,christian,"[[χαίρω, υἱός, θυγάτηρ, ὄνομα, κύριος, ἀγαπάω,...","[[χαίρω, υἱός, θυγάτηρ, ὄνομα, κύριος, ἀγαπάω,...",9,0.001210
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119,tlg0614,Babrius,1,[Fabulae Aesopeae],[tlg0614.tlg001.1st1K-grc2.xml],60616,24068,13978,{'1.5': 1},1.5,pagan,"[[ʼ, ü, ιιι, μῦθος, ἀκούω, βαβρίου, φαβεΐου, γ...","[[], [], [], [], [], [ʼ], [], [], [], [], [], ...",0,0.000000
125,tlg0640,Alciphron,1,[Epistulae],[tlg0640.tlg001.First1K-grc1.xml],36326,15749,6027,"{'1.5': 0.5, '2.5': 0.5}",2.0,pagan,"[[χρηστός, θάλασσα, εἰμί, γαλήνη, ἐστόρεσεν, τ...","[[χρηστός, θάλασσα, εἰμί, γαλήνη, ἐστόρεσεν], ...",0,0.000000
126,tlg0641,Xenophon of Ephesus,1,[Ephesiaca],[tlg0641.tlg001.perseus-grc1.xml],16162,0,1073,"{'1.5': 0.5, '2.5': 0.5}",2.0,,[[]],"[[], [], [], [], [], [], [], [], [], [], [], [...",0,0.000000
127,tlg0643,Hermannus Diels,5,[Preface to Anonymi Londinensis ex Aristotelis...,"[tlg0643.opp001.1st1K-lat1.xml, tlg0643.opp002...",3387,984,1079,{'0.5': 1},0.5,,"[[ʼ, Μᾶρκος, ἀντώνιος, αὐτοκράτωρ, ἀνήρ, δημόσ...","[[], [], [], [], [], [], [], [], [], [], [], [...",0,0.000000


In [169]:
authors_df[authors_df["dikaiosyne_N"]>0].sort_values("date_avr").head(30)

,author_id,author,n_titles,titles,filenames,wordcount,lemmata_wordcount,n_sentences,date_probs,date_avr,provenience,lemmata_lists,lemmatized_sentences,dikaiosyne_N,dikaiosyne_TF
70,tlg0096,Aesop,1,[Fabulae],[tlg0096.tlg002.First1K-grc1.xml],39492,21577,3333,{'-5.5': 1},-5.5,pagan,"[[ἀγαθός, πᾶς, κακός, διώκω, ἀσθενής, εἰμί, οὐ...","[[ἀγαθός, πᾶς, κακός, διώκω, ἀσθενής, εἰμί], [...",2,0.000051
1,tlg0003,Thucydides,1,[The Peloponnesian War],[tlg0003.tlg001.perseus-grc2.xml],150118,71863,6068,{'-4.5': 1},-4.5,pagan,"[[θουκυδίδης, Ἀθηναῖος, συγγράφω, πόλεμος, Πελ...","[[θουκυδίδης, Ἀθηναῖος, συγγράφω, πόλεμος, Πελ...",1,0.000007
19,tlg0016,Herodotus,1,[The Histories],[tlg0016.tlg001.perseus-grc2.xml],184947,94799,10149,{'-4.5': 1},-4.5,pagan,"[[Ἡρόδοτος, Ἁλικαρνασσεύς, ἱστορία, ἀπόδειξις,...","[[Ἡρόδοτος, Ἁλικαρνασσεύς, ἱστορία, ἀπόδειξις,...",8,0.000043
89,tlg0540,Lysias,34,"[On the Murder of Eratosthenes, Funeral Oratio...","[tlg0540.tlg001.perseus-grc2.xml, tlg0540.tlg0...",58875,28254,4258,"{'-4.5': 0.5, '-3.5': 0.5}",-4.0,pagan,"[[πολύς, ποιέω, ἀνήρ, τοιοῦτος, δικαστής, οὗτο...","[[πολύς, ποιέω, ἀνήρ, τοιοῦτος, δικαστής, οὗτο...",2,0.000034
38,tlg0032,Xenophon,14,"[Hellenica, Memorabilia, Economics, Symposium,...","[tlg0032.tlg001.perseus-grc2.xml, tlg0032.tlg0...",312187,150357,18988,"{'-4.5': 0.5, '-3.5': 0.5}",-4.0,pagan,"[[οὗτος, πολύς, ἡμέρα, ὕστερος, ἔρχομαι, Ἀθῆνα...","[[οὗτος, πολύς, ἡμέρα, ὕστερος, ἔρχομαι, Ἀθῆνα...",35,0.000112
122,tlg0627,Hippocrates,52,"[On Ancient Medicine, De aere aquis et locis ,...","[tlg0627.tlg001.perseus-grc1.xml, tlg0627.tlg0...",333443,171332,24456,"{'-4.5': 0.5, '-3.5': 0.5}",-4.0,pagan,"[[ὁπόσος, ἐπιχειρέω, ἰητρικῆς, λέγω, γράφω, ὑπ...","[[ὁπόσος, ἐπιχειρέω, ἰητρικῆς, λέγω, γράφω, ὑπ...",2,0.000006
45,tlg0059,Plato,37,"[Euthyphro, Apology, Crito, Phaedo, Cratylus, ...","[tlg0059.tlg001.perseus-grc1.xml, tlg0059.tlg0...",574294,272931,52384,"{'-4.5': 0.5, '-3.5': 0.5}",-4.0,pagan,"[[ευθ, νέος, Σωκράτης, γίγνομαι, λύκειον, κατα...","[[ευθ], [νέος, Σωκράτης, γίγνομαι, λύκειον, κα...",267,0.000465
7,tlg0010,Isocrates,30,"[Against Euthynus, Against Callimachus, Agains...","[tlg0010.tlg001.perseus-grc2.xml, tlg0010.tlg0...",119155,56497,4681,"{'-4.5': 0.5, '-3.5': 0.5}",-4.0,pagan,"[[πρόφασις, ἀπορέω, λέγω, νικίου, οὗτος, φίλος...","[[πρόφασις, ἀπορέω, λέγω, νικίου, οὗτος], [φίλ...",43,0.000361
17,tlg0014,Demosthenes,63,"[Ὀλυνθιακὸς α΄, Ὀλυνθιακὸς β΄, Ὀλυνθιακὸς γ΄, ...","[tlg0014.tlg001.perseus-grc2.xml, tlg0014.tlg0...",294362,143390,14564,{'-3.5': 1},-3.5,pagan,"[[πολύς, ἀνήρ, Ἀθηναῖος, χρῆμα, αἱρέω, νομίζω,...","[[πολύς, ἀνήρ, Ἀθηναῖος, χρῆμα, αἱρέω, νομίζω,...",11,0.000037
63,tlg0086,Aristotle,35,"[Aristotelis Analytica Priora et Posteriora, D...","[tlg0086.tlg001.1st1K-grc2.xml, tlg0086.tlg002...",840271,367848,60282,{'-3.5': 1},-3.5,pagan,"[[πρῶτος, λέγω, εἰμί, σκέψις, ἀπόδειξις, ἐπιστ...","[[πρῶτος, λέγω, εἰμί, σκέψις, ἀπόδειξις, ἐπιστ...",162,0.000193


# Networks for authors

In [170]:
reload(textnet)

<module 'textnet' from '/home/kasev/ECCE_AGT/scripts/textnet.py'>

In [31]:
%%time
nx_from_sentences = authors_df["lemmatized_sentences"].apply(textnet.network_from_sentences)
nx_from_lemmata = authors_df["lemmata_lists"].apply(textnet.network_from_lemmata_lists)

CPU times: user 24min 50s, sys: 59.6 s, total: 25min 50s
Wall time: 25min 51s


In [171]:
authors_df["G_sents"] = nx_from_sentences
authors_df["G_lems"] = nx_from_lemmata

In [172]:
def ego_network(G):
    try:
        G_ego = textnet.construct_ego_network(G, "δικαιοσύνη", 20, reduced=True)
    except:
        G_ego = nx.Graph()
        G_ego.clear()
    return G_ego

In [173]:
authors_df["dikaiosyne_G_sents"] = authors_df["G_sents"].apply(ego_network)
authors_df["dikaiosyne_G_lems"] = authors_df["G_lems"].apply(ego_network)

In [35]:
list(authors_df.iloc[0]["dikaiosyne_G_sents"].nodes())

['φιλαργυρία',
 'πρέσβυς',
 'οὐδείς',
 'μωμοσκοπεῖται',
 'κύριος',
 'νεώτερος',
 'πᾶς',
 'γινωσκούσας',
 'εἰμί',
 'ετ',
 'οὗτος',
 'ἁμαρτία',
 'ϝος',
 'δικαιοσύνη',
 'ἁγνεία',
 'ἐντολή',
 'θεός',
 'πίστις',
 'εἶμι',
 'κρίσις',
 'προνοοῦντες']

# Terms within sentences

In [36]:
def sentences_with_term(sentences, term, flatlist=False):
    sentences_with_term = [sen for sen in sentences if term in sen]
    if flatlist == True:
        flatlist = [word for sen in sentences_with_term for word in sen]
        flatlist = [word for word in flatlist if word != "δικαιοσύνη"]
        return flatlist
    else: 
        return sentences_with_term
authors_df["dik_sents_flat"] = authors_df["lemmatized_sentences"].apply(lambda cell: sentences_with_term(cell, "δικαιοσύνη", flatlist=True))

In [37]:
def get_tf(lemmata):
    fd_list = list(nltk.FreqDist(lemmata).most_common())
    tf_list = [(tup[0], np.round(tup[1] / len(lemmata), 5)) for tup in fd_list]
    return tf_list

authors_df["dik_sents_TF"] = authors_df["dik_sents_flat"].apply(get_tf)

In [38]:
def term_idf(term, all_sentences):
    idf_raw =  len(all_sentences) / len([sent for sent in all_sentences if term in sent])
    idf_log = math.log(idf_raw)
    return idf_log

def get_tfidf(key_term_lemmata, all_sentences):
    fd_list = list(nltk.FreqDist(key_term_lemmata).most_common())
    tfidf_list = [(tup[0], np.round((tup[1] / len(key_term_lemmata) * term_idf(tup[0], all_sentences)), 5)) for tup in fd_list]
    tfidf_list = sorted(tfidf_list, key=lambda tup: tup[1], reverse=True)
    return tfidf_list

authors_df["dik_sents_TFIDF"] = authors_df.apply(lambda x: get_tfidf(x["dik_sents_flat"], x["lemmatized_sentences"]), axis=1)

In [39]:
authors_df["dik_sents_TFIDF_10"] = authors_df["dik_sents_TFIDF"].apply(lambda cell: ", ".join([tup[0] for tup in cell[:10]]))

In [40]:
authors_df.head(5)

,author_id,author,n_titles,titles,filenames,wordcount,lemmata_wordcount,n_sentences,date_probs,date_avr,...,dikaiosyne_N,dikaiosyne_TF,G_sents,G_lems,dikaiosyne_G_sents,dikaiosyne_G_lems,dik_sents_flat,dik_sents_TF,dik_sents_TFIDF,dik_sents_TFIDF_10
175,tlg1622,Polycarp,1,[Epistula ad Philippenses],[tlg1622.tlg001.1st1K-grc1.xml],1804,951,212,"{'0.5': 0.5, '1.5': 0.5}",1.0,...,8,0.004435,"(πορεύω, εχιστιμετις, ἀπολείπω, ιυδιξιυμ, μέτρ...","(Χριστός, Ἰησοῦς, κύριος, νεκρός, ἐγείρω, θεός...","(φιλαργυρία, πρέσβυς, οὐδείς, μωμοσκοπεῖται, κ...","(ετ, ϝος, εἶμι, ϝοβις, δικαιοσύνη, εἰμί, ἁμαρτ...","[μακάριος, πτωχός, διώκω, εἰμί, βασιλεία, θεάο...","[(εἰμί, 0.04651), (οὗτος, 0.04651), (ἐντολή, 0...","[(οὗτος, 0.19805), (ἐντολή, 0.18466), (πτωχός,...","οὗτος, ἐντολή, πτωχός, διώκω, θεάομαι, ἀδελφός..."
35,tlg0031paul,Paul of Tarsus,7,"[Romans, 1 Corinthians, 2 Corinthians, Galatia...","[tlg0031.tlg006.perseus-grc2.xml, tlg0031.tlg0...",24066,11079,1632,{'0.5': 1},0.5,...,49,0.002036,"(φρόνιμος, ὕβριστος, ἀγανάκτησις, σαργάνη, φρο...","(Χριστός, Ἰησοῦς, κύριος, θεός, χάρις, πατήρ, ...","(λόγος, κύριος, ἀδελφός, ἔχω, εὐαγγέλιον, πίστ...","(Χριστός, Ἰησοῦς, κύριος, θεός, χάρις, πατήρ, ...","[θεός, ἀποκαλύπτω, πίστις, πίστις, γράφω, δίκα...","[(θεός, 0.04675), (πίστις, 0.04156), (νόμος, 0...","[(πίστις, 0.12745), (ἁμαρτία, 0.11192), (νόμος...","πίστις, ἁμαρτία, νόμος, λογίζομαι, θεός, βασιλ..."
156,tlg1271,Clemens Romanus,2,"[Epistula I ad Corinthios, Epistula II ad Cori...","[tlg1271.tlg001.1st1K-grc1.xml, tlg1271.tlg002...",13936,6283,1410,{'0.5': 1},0.5,...,21,0.001507,"(κατασταθέντας, φρόνιμος, ὄρνεον, ὁρωμένοις, φ...","(Χριστός, Ἰησοῦς, κύριος, λέγω, πνεῦμα, ἅγιος,...","(κύριος, ἔχω, πίστις, δίδωμι, ἀγάπη, αἰών, ἐκκ...","(κύριος, λέγω, πνεῦμα, θεός, οὗτος, πᾶς, θέλημ...","[οὗτος, ἄπειμι, εἰρήνη, ἀπολιμπάνω, ἕκαστος, φ...","[(θεός, 0.02871), (πίστις, 0.02392), (γίγνομαι...","[(καυχάομαι, 0.10409), (πίστις, 0.10181), (πύλ...","καυχάομαι, πίστις, πύλη, θεός, δύσις, ἀγάπη, π..."
130,tlg0646,Pseudo-Justinus Martyr,1,[Epistula ad Diognetum],[tlg0646.tlg004.1st1K-grc1.xml],2725,1363,240,"{'2.5': 0.5, '4.5': 0.5}",3.5,...,4,0.001468,"(παρατήρησις, ἀνακοινόω, ἐπίσταμαι, ἀδικούμενο...","(θεός, οὗτος, Χριστιανός, κόσμος, γνῶσις, ξύλο...","(ἄνθρωπος, οὐρανός, γῆ, κρυβῇ, δεῖγμα, δικαιώσ...",(δικαιοσύνη),"[ἐφηδόμενος, ἁμάρτημα, ἀνεχόμενος, ἀδικία, και...","[(ἐκεῖνος, 0.05556), (πολύς, 0.05556), (πῦρ, 0...","[(ἐκεῖνος, 0.24345), (πῦρ, 0.21507), (πολύς, 0...","ἐκεῖνος, πῦρ, πολύς, ἐφηδόμενος, ἁμάρτημα, ἀνε..."
152,tlg1216,Barnabas,1,[The Epistle of Barnabas],[tlg1216.tlg001.perseus-grc2.xml],7440,3434,875,"{'0.5': 0.5, '1.5': 0.5}",1.0,...,9,0.001210,"(πορεύω, πεφυτευμένον, ἐπίσταμαι, ὄρνεον, θεμέ...","(κύριος, λέγω, προφήτης, θεός, οὗτος, γραφή, υ...","(χείρ, ἀγαθός, κύριος, πᾶς, εἰμί, ὁδός, οὗτος,...","(κύριος, λέγω, προφήτης, θεός, οὗτος, γραφή, υ...","[πείθω, οὗτος, σύνοιδα, λαλέω, πολύς, ἐπίσταμα...","[(ὁδός, 0.0411), (κύριος, 0.0274), (θεός, 0.02...","[(ὁδός, 0.17298), (σκότος, 0.13651), (ἀγαθός, ...","ὁδός, σκότος, ἀγαθός, σύνοιδα, ἐπίσταμαι, συνώ..."


In [41]:
tfidf_words = list(set([word for wordlist in authors_df["dik_sents_TFIDF_10"].tolist() for word in wordlist]))
tfidf_words = [word for word in tfidf_words if not re.search("\W", word)]

In [139]:
#set_with_dataframe(ECCE_AGT_overview.add_worksheet("tfidf_words_20201205", 1, 1), pd.DataFrame(tfidf_words))

In [166]:
columns_to_omit = ["lemmatized_sentences", "lemmata_lists", "G_sents", "G_lems", "dikaiosyne_G_sents" , "dikaiosyne_G_lems"]
set_with_dataframe(ECCE_AGT_overview.add_worksheet("authors_df_minimal_20201211", 1,1), authors_df.drop(columns_to_omit, axis=1))

# Individual authors

In [42]:
authors_df[authors_df["author"].str.contains("Isocrates")]["dik_sents_TFIDF_10"].tolist()[0]

'σωφροσύνη, ἀρετή, εὐσέβεια, εἰμί, ἀδικία, σοφία, πολύς, ἔχω, μέγας, οὗτος'

In [43]:
authors_df[authors_df["author"].str.contains("Aristotle")]["dik_sents_TFIDF_10"].tolist()[0]

'δίκαιος, ἀρετή, ἀδικία, ἀνδρεία, σωφροσύνη, ἀγαθός, εἰμί, οἷος, ἄλλος, ἕτερος'

In [121]:
G = authors_df[authors_df["author"].str.contains("Paul of")]["dikaiosyne_G_sents"].tolist()[0]
fig = textnet.draw_2d_network(G)
fig.show()

In [186]:
sddk.write_file("SDAM_data/ECCE_public/G_sents_paul.png",fig, conf)

Your <class 'plotly.graph_objs._figure.Figure'> object has been succefully written as "https://sciencedata.dk/files/SDAM_root/SDAM_data/ECCE_public/G_sents_paul.png"


In [ ]:
reload(textnet)

In [122]:
G = authors_df[authors_df["author"].str.contains("Pauline ")]["dikaiosyne_G_sents"].tolist()[0]
fig = textnet.draw_2d_network(G)
fig.show()

In [188]:
sddk.write_file("SDAM_data/ECCE_public/G_sents_pauline.png",fig, conf)

Your <class 'plotly.graph_objs._figure.Figure'> object has been succefully written as "https://sciencedata.dk/files/SDAM_root/SDAM_data/ECCE_public/G_sents_pauline.png"


# N of shared terms within dik_sents_TFIDF_10

In [48]:
authors_df["dik_sents_TFIDF_10"].tolist()[:5]

['οὗτος, ἐντολή, πτωχός, διώκω, θεάομαι, ἀδελφός, ἐπιτρέπω, προεπεκαλέσασθέ, τὶς, πληρόω',
 'πίστις, ἁμαρτία, νόμος, λογίζομαι, θεός, βασιλεύω, δοῦλος, παρίστημι, πιστεύω, Χριστός',
 'καυχάομαι, πίστις, πύλη, θεός, δύσις, ἀγάπη, πατήρ, ὑπομονή, ἄνθρωπος, Ἀβραάμ',
 'ἐκεῖνος, πῦρ, πολύς, ἐφηδόμενος, ἁμάρτημα, ἀνεχόμενος, συνευδοκέω, ἐλεγχθέντες, ἀνάξιος, καλύπτω',
 'ὁδός, σκότος, ἀγαθός, σύνοιδα, ἐπίσταμαι, συνώδευσεν, ἀναγκάζω, ἀνατελεῖ, προπορεύσεται, περιστελεῖ']

In [66]:
philo_josephus = authors_df[authors_df["author_id"].isin(["tlg0526", "tlg0018"])]

In [67]:
authors_df_c = authors_df[(authors_df["provenience"] == "christian") & (authors_df["date_avr"] < 4)]

In [83]:
# christian + philo + josephus
authors_df_cj = pd.concat([authors_df_c, philo_josephus])

In [84]:
authors_df_cj

,author_id,author,n_titles,titles,filenames,wordcount,lemmata_wordcount,n_sentences,date_probs,date_avr,...,dikaiosyne_N,dikaiosyne_TF,G_sents,G_lems,dikaiosyne_G_sents,dikaiosyne_G_lems,dik_sents_flat,dik_sents_TF,dik_sents_TFIDF,dik_sents_TFIDF_10
175,tlg1622,Polycarp,1,[Epistula ad Philippenses],[tlg1622.tlg001.1st1K-grc1.xml],1804,951,212,"{'0.5': 0.5, '1.5': 0.5}",1.0,...,8,0.004435,"(πορεύω, εχιστιμετις, ἀπολείπω, ιυδιξιυμ, μέτρ...","(Χριστός, Ἰησοῦς, κύριος, νεκρός, ἐγείρω, θεός...","(φιλαργυρία, πρέσβυς, οὐδείς, μωμοσκοπεῖται, κ...","(ετ, ϝος, εἶμι, ϝοβις, δικαιοσύνη, εἰμί, ἁμαρτ...","[μακάριος, πτωχός, διώκω, εἰμί, βασιλεία, θεάο...","[(εἰμί, 0.04651), (οὗτος, 0.04651), (ἐντολή, 0...","[(οὗτος, 0.19805), (ἐντολή, 0.18466), (πτωχός,...","οὗτος, ἐντολή, πτωχός, διώκω, θεάομαι, ἀδελφός..."
35,tlg0031paul,Paul of Tarsus,7,"[Romans, 1 Corinthians, 2 Corinthians, Galatia...","[tlg0031.tlg006.perseus-grc2.xml, tlg0031.tlg0...",24066,11079,1632,{'0.5': 1},0.5,...,49,0.002036,"(φρόνιμος, ὕβριστος, ἀγανάκτησις, σαργάνη, φρο...","(Χριστός, Ἰησοῦς, κύριος, θεός, χάρις, πατήρ, ...","(λόγος, κύριος, ἀδελφός, ἔχω, εὐαγγέλιον, πίστ...","(Χριστός, Ἰησοῦς, κύριος, θεός, χάρις, πατήρ, ...","[θεός, ἀποκαλύπτω, πίστις, πίστις, γράφω, δίκα...","[(θεός, 0.04675), (πίστις, 0.04156), (νόμος, 0...","[(πίστις, 0.12745), (ἁμαρτία, 0.11192), (νόμος...","πίστις, ἁμαρτία, νόμος, λογίζομαι, θεός, βασιλ..."
156,tlg1271,Clemens Romanus,2,"[Epistula I ad Corinthios, Epistula II ad Cori...","[tlg1271.tlg001.1st1K-grc1.xml, tlg1271.tlg002...",13936,6283,1410,{'0.5': 1},0.5,...,21,0.001507,"(κατασταθέντας, φρόνιμος, ὄρνεον, ὁρωμένοις, φ...","(Χριστός, Ἰησοῦς, κύριος, λέγω, πνεῦμα, ἅγιος,...","(κύριος, ἔχω, πίστις, δίδωμι, ἀγάπη, αἰών, ἐκκ...","(κύριος, λέγω, πνεῦμα, θεός, οὗτος, πᾶς, θέλημ...","[οὗτος, ἄπειμι, εἰρήνη, ἀπολιμπάνω, ἕκαστος, φ...","[(θεός, 0.02871), (πίστις, 0.02392), (γίγνομαι...","[(καυχάομαι, 0.10409), (πίστις, 0.10181), (πύλ...","καυχάομαι, πίστις, πύλη, θεός, δύσις, ἀγάπη, π..."
130,tlg0646,Pseudo-Justinus Martyr,1,[Epistula ad Diognetum],[tlg0646.tlg004.1st1K-grc1.xml],2725,1363,240,"{'2.5': 0.5, '4.5': 0.5}",3.5,...,4,0.001468,"(παρατήρησις, ἀνακοινόω, ἐπίσταμαι, ἀδικούμενο...","(θεός, οὗτος, Χριστιανός, κόσμος, γνῶσις, ξύλο...","(ἄνθρωπος, οὐρανός, γῆ, κρυβῇ, δεῖγμα, δικαιώσ...",(δικαιοσύνη),"[ἐφηδόμενος, ἁμάρτημα, ἀνεχόμενος, ἀδικία, και...","[(ἐκεῖνος, 0.05556), (πολύς, 0.05556), (πῦρ, 0...","[(ἐκεῖνος, 0.24345), (πῦρ, 0.21507), (πολύς, 0...","ἐκεῖνος, πῦρ, πολύς, ἐφηδόμενος, ἁμάρτημα, ἀνε..."
152,tlg1216,Barnabas,1,[The Epistle of Barnabas],[tlg1216.tlg001.perseus-grc2.xml],7440,3434,875,"{'0.5': 0.5, '1.5': 0.5}",1.0,...,9,0.001210,"(πορεύω, πεφυτευμένον, ἐπίσταμαι, ὄρνεον, θεμέ...","(κύριος, λέγω, προφήτης, θεός, οὗτος, γραφή, υ...","(χείρ, ἀγαθός, κύριος, πᾶς, εἰμί, ὁδός, οὗτος,...","(κύριος, λέγω, προφήτης, θεός, οὗτος, γραφή, υ...","[πείθω, οὗτος, σύνοιδα, λαλέω, πολύς, ἐπίσταμα...","[(ὁδός, 0.0411), (κύριος, 0.0274), (θεός, 0.02...","[(ὁδός, 0.17298), (σκότος, 0.13651), (ἀγαθός, ...","ὁδός, σκότος, ἀγαθός, σύνοιδα, ἐπίσταμαι, συνώ..."
164,tlg1419,Hermas,1,[The Shepherd of Hermas],[tlg1419.tlg001.1st1K-grc1.xml],30282,14322,2959,{'1.5': 1},1.5,...,33,0.001090,"(ἀνομήσαντα, ὄρνεον, ιλλι, ὑβριζόμενοι, ὁλοτελ...","(κύριος, φημί, ἀκούω, οἰκοδομή, πύργος, εἰμί, ...","(κύριος, οὗτος, τοιοῦτος, ἔχω, ὁράω, μετάνοια,...","(κύριος, φημί, ἀκούω, εἰμί, οὗτος, λέγω, ἔχω, ...","[λέγω, προηγουμένοις, ἐκκλησία, κατορθώσωνται,...","[(ἄγγελος, 0.03333), (ἐργάζομαι, 0.03), (εἰμί,...","[(ἄγγελος, 0.12433), (ἐργάζομαι, 0.12302), (ἀρ...","ἄγγελος, ἐργάζομαι, ἀρετή, ἔργον, ἀλήθεια, πον..."
36,tlg0031pspa,Pauline literature,6,"[Ephesians, Colossians, 2 Thessalonians, 1 Tim...","[tlg0031.tlg010.perseus-grc2.xml, tlg0031.tlg0...",8324,4102,390,{'0.5': 1},0.5,...,8,0.000961,"(ἤρεμος, ἐπίσταμαι, μάχη, ἅλυσις, Ἰουδαϊκός, κ...","(Χριστός, Ἰησοῦς, κύριος, θεός, πατήρ, ἀγαθός,...","(ἄνθρωπος, γνωρίζω, λόγος, κύριος, πᾶς, χάρις,...","(διώκω, φεύγω, δικαιοσύν

In [89]:
edges_tuples = []
for author1 in authors_df_cj["author"].tolist():
    author1_words = authors_df_cj.loc[authors_df_cj["author"]==author1, "dik_sents_TFIDF_10"].values[0].split()
    for author2 in authors_df_cj["author"].tolist():
        if author2 != author1:
            author2_words = authors_df_cj.loc[authors_df_cj["author"]==author2, "dik_sents_TFIDF_10"].values[0].split()
            shared = len(set(author1_words) & set(author2_words))
            if shared > 1:
                edges_tuples.append((author1, author2, shared))

In [90]:
G = nx.Graph()
G.clear()
G.add_weighted_edges_from(edges_tuples)
for (u, v, wt) in G.edges.data('weight'):
    G[u][v]["weight"] = int(wt)
total_weight = sum([int(n) for n in nx.get_edge_attributes(G, "weight").values()])
for (u, v) in G.edges:
    G[u][v]["norm_weight"] = round((G[u][v]["weight"] / total_weight), 5)
    G[u][v]["distance"] = round(1 / (G[u][v]["weight"]), 5)
    G[u][v]["norm_distance"] = round(1 / (G[u][v]["norm_weight"] ), 5)
G.remove_edges_from(nx.selfloop_edges(G))

In [116]:
reload(textnet)
textnet.draw_2d_network(G)

In [53]:
reload(textnet)

<module 'textnet' from '/home/kasev/ECCE_AGT/scripts/textnet.py'>

In [ ]:
sddk.write_file("SDAM_data/ECCE_public/authors_network_10tfidf.png", fig, conf)

In [726]:
fig.write_image("../figures/authors_network_10tfidf.png")

In [117]:
G_paul = textnet.construct_ego_network(G, "Paul of Tarsus", 10, reduced=True)

In [119]:
reload(textnet)

<module 'textnet' from '/home/kasev/ECCE_AGT/scripts/textnet.py'>

In [120]:
fig = textnet.draw_2d_network(G_paul)
fig.show()

In [756]:
fig.write_image("../figures/paul_egonet_10tfidf.png")

In [733]:
nx.to_pandas_adjacency(G_paul).astype(int)

,Paul of Tarsus,Clemens Romanus,Origenes,Theophilus Antiochenus,Eusebius,Epiphanius,Clement of Alexandria,Hermas,Athenagoras,Johnannine literature,Basil
Paul of Tarsus,0,2,2,3,2,5,0,0,0,0,0
Clemens Romanus,2,0,0,0,0,0,0,0,0,0,0
Origenes,2,0,0,0,0,0,0,0,0,3,0
Theophilus Antiochenus,3,0,0,0,0,0,0,0,0,0,2
Eusebius,2,0,0,0,0,0,0,0,0,0,0
Epiphanius,5,0,0,0,0,0,3,2,2,0,0
Clement of Alexandria,0,0,0,0,0,3,0,0,0,0,0
Hermas,0,0,0,0,0,2,0,0,0,0,0
Athenagoras,0,0,0,0,0,2,0,0,0,0,0
Johnannine literature,0,0,3,0,0,0,0,0,0,0,0


#  vectorization based on tfidf values

In [734]:
#tup_to_dict 
tfidfs_df_raw = pd.DataFrame([dict(tfidfs) for tfidfs in authors_df["dik_sents_TFIDF"].tolist()])
tfidfs_df_raw

,οὗτος,ἐντολή,πτωχός,διώκω,θεάομαι,ἀδελφός,ἐπιτρέπω,προεπεκαλέσασθέ,πληρόω,ἐκφέρω,...,ὑπερβάλλω,προειρημένου,παρακολουθέω,Ἰωσήφ,ἐγγίων,τελευτάω,ἀπεργάζομαι,τέσσαρες,ἡττηθέντα,κᾀκεῖνον
0,0.18466,0.18466,0.12457,0.12457,0.12457,0.12457,0.12457,0.12457,0.12457,0.12457,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.00714,NaN,NaN,0.03494,0.01016,NaN,NaN,NaN,0.01269,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.03078,NaN,NaN,0.02575,0.02113,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0.04409,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,0.04574,0.03691,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,0.07851,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,0.01886,NaN,NaN,NaN,NaN,NaN,NaN,0.02423,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,0.01381,0.01541,0.03197,NaN,NaN,NaN,NaN,NaN,0.01578,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [757]:
selected_columns = []
for column in tfidfs_df_raw.columns:
    if len(tfidfs_df_raw[tfidfs_df_raw[column].notnull()]) > 1:
        selected_columns.append(column)
len(selected_columns)

861

In [758]:
tfidfs_df_filtered = tfidfs_df_raw[selected_columns]
tfidfs_df_filtered.fillna(0, inplace=True)

In [759]:
tfidfs_df_filtered

,οὗτος,ἐντολή,πτωχός,διώκω,θεάομαι,ἀδελφός,πληρόω,ἐκφέρω,ὅπλον,πρῶτος,...,τύραννος,τιμωρία,θεοφιλής,ὁμολογήσαντα,θελήσαντα,δουλεία,πατριάρχης,δύο,Μαρία,ἐγείρω
0,0.18466,0.18466,0.12457,0.12457,0.12457,0.12457,0.12457,0.12457,0.12457,0.12457,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
1,0.00714,0.00000,0.00000,0.03494,0.01016,0.00000,0.01269,0.00000,0.04660,0.00000,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
2,0.03078,0.00000,0.00000,0.02575,0.02113,0.00000,0.00000,0.00000,0.00000,0.00000,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
3,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
4,0.04409,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
5,0.04574,0.03691,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.01386,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
6,0.00000,0.00000,0.00000,0.07851,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
7,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
8,0.00000,0.01886,0.00000,0.00000,0.00000,0.00000,0.02423,0.00000,0.00000,0.01361,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
9,0.01381,0.01541,0.03197,0.00000,0.00000,0.00000,0.01578,0.00000,0.00000,0.00000,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000


# Tfidfs with cosine similarity

In [760]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import TruncatedSVD 
from sklearn.metrics.pairwise import cosine_similarity

In [761]:
tfidfs_sim_mat = cosine_similarity(tfidfs_df_filtered.to_numpy()).round(5)
tfidfs_dist_mat = 1 - cosine_similarity(tfidfs_df_filtered.to_numpy())
tfidfs_dist_mat = tfidfs_dist_mat.round(5)

In [762]:
authors = authors_df["author"].tolist()

In [763]:
tfidfs_sim_mat_df = pd.DataFrame(tfidfs_sim_mat, columns=authors, index=authors)
tfidfs_sim_mat_df

,Polycarp,Paul of Tarsus,Clemens Romanus,Pseudo-Justinus Martyr,Barnabas,Hermas,Pauline literature,Didache,GNT rest,Justinus Martyr,...,Luke (the evangelist),Ignatius Antiochenus,Basil,Maximus of Tyre,Hippolytus,Irenaeus,Tatianus,Martyrium Polycarpi,Mark,Dionysius Corinthius
Polycarp,1.00000,0.22181,0.23814,0.00707,0.07773,0.22399,0.12430,0.03068,0.17802,0.13459,...,0.09773,0.13008,0.04870,0.04291,0.02438,0.04281,0.0,0.0,0.0,0.0
Paul of Tarsus,0.22181,1.00000,0.36451,0.20356,0.18825,0.25099,0.21144,0.03965,0.35972,0.27068,...,0.18380,0.02245,0.20670,0.08929,0.08067,0.04672,0.0,0.0,0.0,0.0
Clemens Romanus,0.23814,0.36451,1.00000,0.08265,0.20547,0.31172,0.29054,0.06315,0.29542,0.30911,...,0.24766,0.00000,0.19421,0.05100,0.07233,0.10855,0.0,0.0,0.0,0.0
Pseudo-Justinus Martyr,0.00707,0.20356,0.08265,1.00000,0.05827,0.13644,0.08101,0.00000,0.16769,0.13493,...,0.03768,0.00000,0.12313,0.09990,0.03972,0.05339,0.0,0.0,0.0,0.0
Barnabas,0.07773,0.18825,0.20547,0.05827,1.00000,0.19986,0.15717,0.37385,0.18561,0.52461,...,0.16731,0.00000,0.08794,0.05815,0.01787,0.06708,0.0,0.0,0.0,0.0
Hermas,0.22399,0.25099,0.31172,0.13644,0.19986,1.00000,0.25905,0.12739,0.27988,0.24091,...,0.22410,0.00000,0.19954,0.08903,0.09794,0.13496,0.0,0.0,0.0,0.0
Pauline literature,0.12430,0.21144,0.29054,0.08101,0.15717,0.25905,1.00000,0.11154,0.24275,0.23870,...,0.15668,0.00000,0.18082,0.02560,0.04015,0.02968,0.0,0.0,0.0,0.0
Didache,0.03068,0.03965,0.06315,0.00000,0.37385,0.12739,0.11154,1.00000,0.02730,0.11109,...,0.08341,0.00000,0.05273,0.00000,0.00000,0.00000,0.0,0.0,0.0,0.0
GNT rest,0.17802,0.35972,0.29542,0.16769,0.18561,0.27988,0.24275,0.02730,1.00000,0.32517,...,0.22292,0.05269,0.15571,0.03767,0.04363,0.03382,0.0,0.0,0.0,0.0
Justinus Martyr,0.13459,0.27068,0.30911,0.13493,0.52461,0.24091,0.23870,0.11109,0.32517,1.00000,...,0.25626,0.02827,0.23503,0.09629,0.09710,0.08615,0.0,0.0,0.0,0.0


In [ ]:
set_with_dataframe(ECCE_AGT_overview.add_worksheet("tfidfs_sim", 1, 1), tfidfs_sim_mat_df, include_index=True)

In [764]:
G = nx.from_numpy_matrix(tfidfs_sim_mat)

authors_dict = dict(zip(range(len(authors)), authors))
G = nx.relabel_nodes(G, authors_dict)

total_weight = sum([int(n) for n in nx.get_edge_attributes(G, "weight").values()])
for (u, v) in G.edges:
    G[u][v]["norm_weight"] = round((G[u][v]["weight"] / total_weight), 5)
    G[u][v]["distance"] = round(1 / (G[u][v]["weight"]), 5)
    G[u][v]["norm_distance"] = round(1 / (G[u][v]["norm_weight"] ), 5)

edges_to_remove = []
for edge in G.edges:
    if edge[0] == edge[1]:
        edges_to_remove.append(edge)
G.remove_edges_from(edges_to_remove)

In [765]:
sim_threshold = 0.3
edges_to_remove = []
for edge in G.edges:
    if G[edge[0]][edge[1]]["weight"] < sim_threshold:
        edges_to_remove.append(edge)
G.remove_edges_from(edges_to_remove)

G.remove_nodes_from(list(nx.isolates(G)))

In [766]:
G.edges.data()

EdgeDataView([('Paul of Tarsus', 'Clemens Romanus', {'weight': 0.36451, 'norm_weight': 0.01402, 'distance': 2.74341, 'norm_distance': 71.32668}), ('Paul of Tarsus', 'GNT rest', {'weight': 0.35972, 'norm_weight': 0.01384, 'distance': 2.77994, 'norm_distance': 72.25434}), ('Paul of Tarsus', 'Clement of Alexandria', {'weight': 0.374, 'norm_weight': 0.01438, 'distance': 2.6738, 'norm_distance': 69.54103}), ('Paul of Tarsus', 'Origenes', {'weight': 0.46308, 'norm_weight': 0.01781, 'distance': 2.15945, 'norm_distance': 56.14823}), ('Paul of Tarsus', 'Theophilus Antiochenus', {'weight': 0.32512, 'norm_weight': 0.0125, 'distance': 3.07579, 'norm_distance': 80.0}), ('Paul of Tarsus', 'Eusebius', {'weight': 0.39914, 'norm_weight': 0.01535, 'distance': 2.50539, 'norm_distance': 65.14658}), ('Paul of Tarsus', 'Epiphanius', {'weight': 0.51618, 'norm_weight': 0.01985, 'distance': 1.93731, 'norm_distance': 50.37783}), ('Clemens Romanus', 'Hermas', {'weight': 0.31172, 'norm_weight': 0.01199, 'distance

In [768]:
fig = textnet.draw_2d_network(G)
fig.show()

In [745]:
fig = textnet.draw_2d_network(G)
fig.show()

In [746]:
fig.write_image("../figures/authors_network_cos-sim-tfidf.png")

In [747]:
G_paul = textnet.construct_ego_network(G, "Paul of Tarsus", 10, reduced=True)
fig = textnet.draw_2d_network(G_paul)
fig.show()

In [749]:
fig.write_image("../figures/paul_egonet_cos-sim-tfidf.png")